In [13]:
import requests
import bs4
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

In [14]:
url_inicial = 'https://historiamexicana.colmex.mx/index.php/RHM/issue/view/320'
url_root = 'https://historiamexicana.colmex.mx/index.php/RHM/issue/view/320'
r=requests.get(url_inicial)

In [15]:
soup = BeautifulSoup(r.text, 'html.parser')

In [16]:
lista_articulo = soup.find_all('div', class_='obj_article_summary')
links_libros = [x.find('a').get('href') for x in lista_articulo]
links_libros = [urljoin (url_root, i) for i in links_libros]
links_libros

[]

In [ ]:
def get_url_items(soup,url):
    lista_articulo = soup.find_all('div', class_='obj_article_summary')
    links_libros = [x.find('a').get('href') for x in lista_articulo]
    links_libros = [urljoin (url_root, i) for i in links_libros]
    return links_libros

In [ ]:
links_items=[]
i = 0
while i<1:
    i+=1
    print(f'Estoy en la pagina {url_inicial}')
    r_pag = requests.get(url_inicial)
    s_p = BeautifulSoup(r_pag.text, 'html.parser')
    links = get_url_items(s_p, url_inicial)
    links_items.append(links)
    next_a = s_p.select('div.cmp_pagination > a')
    if not next_a or not next_a[0].get('href'):
        break
        url_inicial = urljoin(url_inicial, next_a[0].get('href'))

In [ ]:
list_scraper=[]
for i in links_items:
    for j in i:
        list_scraper.append(j)
len(list_scraper)

In [ ]:
uno=list_scraper[0]
r_item=requests.get(uno)
s_item=BeautifulSoup(r_item.text, 'html.parser')

In [ ]:
#funcion para iniciar el escraper de cada libro
def scraper_book(url):
    content_book={}
    r=requests.get(url)
    tire='Historia Mexicana'
    a='IV. Humanidades y Ciencias de la Conducta'
    tem='Historia'
    url='http://historiamexicana.colmex.mx/index.php/RHM/index'
    s_item=BeautifulSoup(r.text,'html.parser')
    #titulo de revista
    try:
        titulo=tire
        content_book['Titulo de revista']=tire
    except AttributeError:
        content_book['Titulo de revista']=None
    #area
    try:
        area=a
        content_book['Área']=a
    except AttributeError:
        content_book['Área']=None
    #tematica
    try:
        tema=tem
        content_book['Temática']=tem
    except AttributeError:
        content_book['Temática']=None
    #titulo libro
    try:
        titu=s_item.find('h1', class_='jatsParser__meta-title').get_text(strip=True)
        content_book['Titulo del libro']=titu
    except AttributeError:
        content_book['Titulo del libro']=None
    #abstract
    try:
        abst=s_item.find('a', rel='license').get_text(strip=True)
        content_book['Abstract']=abst
    except AttributeError:
        content_book['Abstract']=None
    #resumen
    try:
        resu=s_item.find('p').get_text(strip=True)
        content_book['Resumen']=resu
    except AttributeError:
        content_book['Resumen']=None
    #Link incial
    try:
        inicial=url
        content_book['Link inicial']=url
    except AttributeError:
        content_book['Link inicial']=None
    #link articulo
    try:
        link=s_item.find('div', class_='jatsParser__cover-wrapper').get('href')
        content_book['Link articulo']=urljoin(url_root, link)
    except AttributeError:
        content_book['Link articulo']=None
    return content_book

In [ ]:
list_scraper=list_scraper[0:94]
datos_book=[]
for idx, i in enumerate(list_scraper):
    print(f'estas escrapeando la pag {idx}')
    datos_book.append(scraper_book(i))

In [ ]:
df_catalogo=pd.DataFrame(datos_book)
df_catalogo

In [ ]:
df_catalogo.to_csv('Revista11.csv')